## LRUCache
---

In [2]:
use std::cell::RefCell;
use std::collections::HashMap;
use std::rc::{Rc, Weak};

In [3]:
#[derive(Debug)]
struct Node<T: PartialEq + PartialOrd + Clone> {
    label: String,
    val: T,
    next: Option<Rc<RefCell<Node<T>>>>,
    prior: Option<Weak<RefCell<Node<T>>>>,
}

In [4]:
impl<T: PartialEq + PartialOrd + Clone> Node<T> {
    fn new(label: String, val: T) -> Self {
        Node { label: label, val: val, next: None, prior: None }
    }
}

In [5]:
#[derive(Debug)]
struct LinkedList<T: PartialEq + PartialOrd + Clone> {
    head: Option<Rc<RefCell<Node<T>>>>,
    tail: Option<Weak<RefCell<Node<T>>>>,
}

In [6]:
impl<T: PartialEq + PartialOrd + Clone> LinkedList<T> {
    fn new() -> Self {
        LinkedList { head: None, tail: None }
    }
    fn push(&mut self, label: String, val: T) -> &Self {
        let v1 = Rc::new(RefCell::new(Node::new(label, val)));
        if self.head.is_none() {
            self.head = Some(v1.clone());
            self.tail = Some(Rc::downgrade(&v1));
        } else {
            let v2 = self.head.take().unwrap();
            v1.borrow_mut().next = Some(v2.clone());
            v2.borrow_mut().prior = Some(Rc::downgrade(&v1));
            self.head = Some(v1);
        }
        self
    }
}

In [7]:
#[derive(Debug)]
struct LRUCache<T: PartialEq + PartialOrd + Clone> {
    llist: LinkedList<T>,
    hmap: HashMap<String, Weak<RefCell<Node<T>>>>,
}

In [8]:
impl<T: PartialEq + PartialOrd + Clone> LRUCache<T> {
    fn new() -> Self {
        LRUCache {
            llist: LinkedList::<T>::new(),
            hmap: HashMap::<String, Weak<RefCell<Node<T>>>>::new()
        }
    }
    fn push(&mut self, label: &str, val: T) -> &Self {
        self.llist.push(label.clone().to_owned(), val);
        let hval = Rc::downgrade(&self.llist.head.as_ref().unwrap().clone());
        self.hmap.insert(label.to_owned(), hval);
        self
    }
}

In [9]:
let mut q = LRUCache::<i32>::new();
q

LRUCache { llist: LinkedList { head: None, tail: None }, hmap: {} }

In [10]:
q.push("aaa", 77)

LRUCache { llist: LinkedList { head: Some(RefCell { value: Node { label: "aaa", val: 77, next: None, prior: None } }), tail: Some((Weak)) }, hmap: {"aaa": (Weak)} }

In [11]:
q.push("bbb", 44)

LRUCache { llist: LinkedList { head: Some(RefCell { value: Node { label: "bbb", val: 44, next: Some(RefCell { value: Node { label: "aaa", val: 77, next: None, prior: Some((Weak)) } }), prior: None } }), tail: Some((Weak)) }, hmap: {"aaa": (Weak), "bbb": (Weak)} }

In [12]:
q.push("ccc", 88)

LRUCache { llist: LinkedList { head: Some(RefCell { value: Node { label: "ccc", val: 88, next: Some(RefCell { value: Node { label: "bbb", val: 44, next: Some(RefCell { value: Node { label: "aaa", val: 77, next: None, prior: Some((Weak)) } }), prior: Some((Weak)) } }), prior: None } }), tail: Some((Weak)) }, hmap: {"ccc": (Weak), "aaa": (Weak), "bbb": (Weak)} }